In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
from itertools import combinations
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve
import missingno as msno
import ast
from collections import Counter
import statsmodels.regression.recursive_ls as rls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyvis.network import Network

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_pickle('../data/df_movie.pkl')
df.head()

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Main_genre,Sec_Genre,Movie_rating,Producer,...,Actor_name,Actor_age_release,Inflation Factor for 2023,2023 valued revenue,Avg_revenue_per_film_at_release,Longevity,Number_of_film_at_release,Avg_rating_per_film_at_release,First_film,Actor_main_genre
403088,The Fox and the Hound,1981.0,63456988.0,83.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Adventure,Children's/Family,NaN,NaN,...,'Squeeks' the Caterpillar,NaN,3.38,2.144846e+08,6.345699e+07,0.0,1,NaN,True,Adventure
400285,Miss March,2009.0,4591629.0,90.0,"{""/m/05zjd"": ""Portuguese Language"", ""/m/02h40l...","{""/m/09c7w0"": ""United States of America""}",Road movie,Sex comedy,NaN,NaN,...,40 Glocc,29.0,1.43,6.566029e+06,4.591629e+06,0.0,1,NaN,True,Road movie
71882,Get Rich or Die Tryin',2005.0,46442528.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Crime Fiction,Hip hop movies,5.4,Jimmy Iovine,...,50 Cent,30.0,1.58,7.337919e+07,4.644253e+07,0.0,1,5.400000,True,Crime Fiction
419847,Home of the Brave,2006.0,499620.0,106.0,"{""/m/0jzc"": ""Arabic Language"", ""/m/02h40lc"": ""...","{""/m/09c7w0"": ""United States of America"", ""/m/...",Drama,War film,5.6,NaN,...,50 Cent,31.0,1.53,7.644186e+05,2.347107e+07,1.0,2,5.500000,False,Crime Fiction
126916,Righteous Kill,2008.0,76747202.0,100.0,"{""/m/06b_j"": ""Russian Language"", ""/m/02h40lc"":...","{""/m/09c7w0"": ""United States of America""}",Thriller,Crime Fiction,6.0,NaN,...,50 Cent,33.0,1.43,1.097485e+08,4.122978e+07,3.0,3,5.666667,False,Crime Fiction


In [3]:
df1 = pd.read_pickle('../data/df_pairs.pkl')
df1.head()

,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre
0,'Til There Was You,1997.0,"(Alice Drummond, Christine Ebersole)",3525125.0,4.8,Alice Drummond,Christine Ebersole,25.0,3,3.612798e+07,False,False,0,False,None
1,'Til There Was You,1997.0,"(Alice Drummond, Craig Bierko)",3525125.0,4.8,Alice Drummond,Craig Bierko,37.0,5,1.416641e+07,False,False,0,False,None
2,'Til There Was You,1997.0,"(Alice Drummond, Dylan McDermott)",3525125.0,4.8,Alice Drummond,Dylan McDermott,34.0,1,1.315054e+07,False,False,0,False,None
3,'Til There Was You,1997.0,"(Alice Drummond, Jeanne Tripplehorn)",3525125.0,4.8,Alice Drummond,Jeanne Tripplehorn,36.0,2,1.241967e+08,False,False,0,False,None
4,'Til There Was You,1997.0,"(Alice Drummond, Jennifer Aniston)",3525125.0,4.8,Alice Drummond,Jennifer Aniston,41.0,3,4.310507e+07,False,False,0,False,None


In [4]:
df2 = df1.copy()

#Filter the years to have only the films from 1980 to 1985 first
df2 = df2[(df2['Movie_release'] >= 1980) & (df2['Movie_release'] <= 2020)]


# Step 1: Create a mapping DataFrame for 'Actor_pairs' to 'Actor1', 'Actor2', and 'Genre'
actor_pairs_mapping = df2[['Actor_pairs', 'Actor1', 'Actor2', 'Genre']].drop_duplicates()

# Step 2: Grouping by 'Actor_pairs' and calculating the required metrics along with including 'Genre'
grouped_df = df2.groupby('Actor_pairs').agg(
    Average_Movie_revenue=pd.NamedAgg(column='Movie_revenue', aggfunc='mean'),
    Average_Movie_rating=pd.NamedAgg(column='Movie_rating', aggfunc='mean'),
    Count=pd.NamedAgg(column='Movie_name', aggfunc='count')
)

# Reset index in the grouped DataFrame
grouped_df.reset_index(inplace=True)

# Step 3: Merge the aggregated DataFrame with the mapping DataFrame
# Note: The merge may result in multiple rows per actor pair if they have multiple genres.
final_df = pd.merge(grouped_df, actor_pairs_mapping, on='Actor_pairs')

final_df

,Actor_pairs,Average_Movie_revenue,Average_Movie_rating,Count,Actor1,Actor2,Genre
0,"(50 Cent, Adewale Akinnuoye-Agbaje)",46442528.0,5.4,1,50 Cent,Adewale Akinnuoye-Agbaje,None
1,"(50 Cent, Al Pacino)",76747202.0,6.0,1,50 Cent,Al Pacino,None
2,"(50 Cent, Alan Blumenfeld)",76747202.0,6.0,1,50 Cent,Alan Blumenfeld,Crime Fiction
3,"(50 Cent, Alan Rosenberg)",76747202.0,6.0,1,50 Cent,Alan Rosenberg,None
4,"(50 Cent, Ambyr Childers)",2566717.0,5.6,1,50 Cent,Ambyr Childers,Crime Fiction
...,...,...,...,...,...,...,...
474803,"(Zhenwei Wang, Zhiheng Wang)",359126022.0,6.2,1,Zhenwei Wang,Zhiheng Wang,Action/Adventure
474804,"(Zoe Saldana, Zulay Henao)",70587268.0,6.2,1,Zoe Saldana,Zulay Henao,None
474805,"(Zoe Saldana, Óscar Jaenada)",29379723.0,6.2,1,Zoe Saldana,Óscar Jaenada,Thriller
474806,"(Zoe Saldana, Željko Ivanek)",11494838.0,8.6,1,Zoe Saldana,Željko Ivanek,Thriller


In [5]:
from sklearn.preprocessing import MinMaxScaler


# Filter to only keep real duos
duos = final_df[final_df['Count'] >=3]

# Creating a copy of the DataFrame slice
duos_standardized = duos.copy()

# Initialize the StandardScaler
standard_scaler = MinMaxScaler()

# Selecting the columns to be normalized
cols_to_normalize = ['Average_Movie_revenue', 'Average_Movie_rating']

# Applying normalization to the selected columns
duos_standardized[cols_to_normalize] = standard_scaler.fit_transform(duos_standardized[cols_to_normalize])

def round_down_to_nearest_05(number):
    return np.floor(number / 0.05) * 0.05

duos_standardized['Average_Movie_revenue'] = duos_standardized['Average_Movie_revenue'].apply(round_down_to_nearest_05)


rating_stand = duos_standardized.sort_values(by=["Average_Movie_rating","Average_Movie_revenue"], ascending= False)


revenue_stand = duos_standardized.copy()
revenue_stand = duos_standardized.sort_values(by=["Average_Movie_revenue","Average_Movie_rating"], ascending= False)


rating_stand.reset_index(drop=True, inplace=True)
rating_stand['rank'] = rating_stand.index + 1   # Adding 1 to start the ranking from 1

revenue_stand.reset_index(drop=True, inplace=True)
revenue_stand['rank'] = revenue_stand.index + 1   # Adding 1 to start the ranking from 1


for i in range(1, len(rating_stand)):
    # Check if the current row has the same speed and mass as the previous row
    if (rating_stand.loc[i, 'Average_Movie_revenue'] == rating_stand.loc[i-1, 'Average_Movie_revenue']) and (rating_stand.loc[i, 'Average_Movie_rating'] == rating_stand.loc[i-1, 'Average_Movie_rating']):
        # Update the rank to be the same as the previous row
        rating_stand.loc[i, 'rank'] = rating_stand.loc[i-1, 'rank']
    
for i in range(1, len(revenue_stand)):
    # Check if the current row has the same speed and mass as the previous row
    if (revenue_stand.loc[i, 'Average_Movie_revenue'] == revenue_stand.loc[i-1, 'Average_Movie_revenue']) and (revenue_stand.loc[i, 'Average_Movie_rating'] == rating_stand.loc[i-1, 'Average_Movie_rating']):
        # Update the rank to be the same as the previous row
        revenue_stand.loc[i, 'rank'] = revenue_stand.loc[i-1, 'rank']

length = len(rating_stand)

rating_stand['rank_ratio']  = (length - (rating_stand['rank']-1))/ length
revenue_stand['rank_ratio']  = (length - (revenue_stand['rank']-1))/ length

# Function to transform x to y and create a tuple
def transform(x):
    if x >= 0.5:
        y = (x - 0.5) * 2  
        return (0, y, 0.3)
    else:
        y = np.abs((x - 0.5) * 2)
        return (y, 0, 0.3)

# Apply the transformation
rating_stand['Color'] = rating_stand['rank_ratio'].apply(transform)
revenue_stand['Color'] = revenue_stand['rank_ratio'].apply(transform)

rating_stand

,Actor_pairs,Average_Movie_revenue,Average_Movie_rating,Count,Actor1,Actor2,Genre,rank,rank_ratio,Color
0,"(Andy Serkis, Billy Boyd)",1.00,1.000000,3,Andy Serkis,Billy Boyd,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
1,"(Andy Serkis, Cate Blanchett)",1.00,1.000000,3,Andy Serkis,Cate Blanchett,None,1,1.000000,"(0, 1.0, 0.3)"
2,"(Andy Serkis, Christopher Lee)",1.00,1.000000,3,Andy Serkis,Christopher Lee,None,1,1.000000,"(0, 1.0, 0.3)"
3,"(Andy Serkis, David Wenham)",1.00,1.000000,3,Andy Serkis,David Wenham,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
4,"(Andy Serkis, Dominic Monaghan)",1.00,1.000000,3,Andy Serkis,Dominic Monaghan,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
...,...,...,...,...,...,...,...,...,...,...
2172,"(Loretta Devine, Regina Hall)",0.00,0.116906,4,Loretta Devine,Regina Hall,None,2172,0.002756,"(0.994487827285255, 0, 0.3)"
2173,"(Anthony Anderson, Loretta Devine)",0.00,0.115108,3,Anthony Anderson,Loretta Devine,Crime Fiction,2174,0.001837,"(0.99632521819017, 0, 0.3)"
2174,"(David Mann, Tamela Mann)",0.05,0.079137,3,David Mann,Tamela Mann,None,2175,0.001378,"(0.9972439136426274, 0, 0.3)"
2175,"(David Mann, Tyler Perry)",0.05,0.079137,3,David Mann,Tyler Perry,None,2175,0.001378,"(0.9972439136426274, 0, 0.3)"


In [6]:
import networkx as nx
import community as community_louvain
import numpy as np
from collections import defaultdict

# Create a new NetworkX graph
G = nx.Graph()

# Add nodes and edges from the DataFrame
for _, row in rating_stand.iterrows():
    G.add_edge(row['Actor1'], row['Actor2'], weight=row['Count'])

# Detect communities (clusters) using the Louvain method
partition = community_louvain.best_partition(G, weight='weight')

# Group nodes by their cluster
clusters = defaultdict(list)
for node, cluster_id in partition.items():
    clusters[cluster_id].append(node)

# Filter clusters with more than 5 nodes
large_clusters = {k: v for k, v in clusters.items() if len(v) > 5}

# Analysis for each large cluster
cluster_averages = {}

for cluster_id, nodes in large_clusters.items():
    # Extract rows from DataFrame that belong to the current cluster
    cluster_rows = rating_stand[(rating_stand['Actor1'].isin(nodes)) | (rating_stand['Actor2'].isin(nodes))]

    # Calculate average rank for the cluster
    average_rank = cluster_rows['rank'].mean() if not cluster_rows.empty else None

    # Store the results
    cluster_averages[cluster_id] = {
        'actors': nodes,
        'average_rank': average_rank
    }

# Sort clusters by average rank in ascending order
sorted_cluster_averages = dict(sorted(cluster_averages.items(), key=lambda item: item[1]['average_rank']))

# List to store the cluster analysis results
cluster_analysis = []

# Add cluster analysis results to the list
for cluster_id, info in sorted_cluster_averages.items():
    cluster_analysis.append({
        'cluster_id': cluster_id,
        'actors': info['actors'],
        'average_rank': info['average_rank']
    })

# Print the cluster analysis results
for cluster in cluster_analysis:
    print(f"Cluster {cluster['average_rank']}: {cluster['actors']}")

Cluster 386.95714285714286: ['Andy Serkis', 'Billy Boyd', 'Cate Blanchett', 'Christopher Lee', 'David Wenham', 'Dominic Monaghan', 'Elijah Wood', 'Hugo Weaving', 'Ian McKellen', 'John Rhys-Davies', 'Liv Tyler', 'Orlando Bloom', 'Sala Baker', 'Sean Astin', 'Sean Bean', 'Viggo Mortensen', 'Peter Jackson', 'Ian Holm', 'Brad Dourif', 'Hugh Jackman', 'Rebecca Romijn', 'Aaron Stanford', 'Anna Paquin', 'Bruce Davison', 'Famke Janssen', 'Halle Berry', 'James Marsden', 'Patrick Stewart', 'Jacek Koman', 'Richard Roxburgh', 'Brent Spiner', 'Gates McFadden', 'Jonathan Frakes', 'LeVar Burton', 'Marina Sirtis', 'Michael Dorn', 'Danny Mann', 'Miriam Margolyes', 'Magda Szubanski', 'Gretchen Mol']
Cluster 437.3171206225681: ['Gary Oldman', 'Alfred Enoch', 'Timothy Spall', 'Bonnie Wright', 'Daniel Radcliffe', 'David Bradley', 'Devon Murray', 'Emma Watson', 'Geraldine Somerville', 'James Phelps', 'Jamie Waylett', 'Joshua Herdman', 'Maggie Smith', 'Mark Williams', 'Matthew David Lewis', 'Michael Gambon', 

In [7]:
import scipy.stats as stats
import numpy as np

# Characteristics to analyze
characteristics = ['Age_difference', 'Film_count_difference', 'Average_revenue_difference', 'Number_of_films_together', 'Same_genre']

# Function to calculate the average of a characteristic for a cluster
def calculate_average_for_cluster(cluster, df, characteristic):
    cluster_rows = df[(df['Actor1'].isin(cluster)) | (df['Actor2'].isin(cluster))]
    return cluster_rows[characteristic].mean()

# Group clusters based on their average rank
median_rank = np.median([info['average_rank'] for info in cluster_averages.values()])
high_rank_clusters = [info['actors'] for info in cluster_averages.values() if info['average_rank'] > median_rank]
low_rank_clusters = [info['actors'] for info in cluster_averages.values() if info['average_rank'] <= median_rank]

# Initialize a list to store T-test results
t_test_results = []

# Perform T-tests for each characteristic
for characteristic in characteristics:
    high_rank_averages = [calculate_average_for_cluster(cluster, df1, characteristic) for cluster in high_rank_clusters]
    low_rank_averages = [calculate_average_for_cluster(cluster, df1, characteristic) for cluster in low_rank_clusters]

    # Perform a T-test
    t_stat, p_value = stats.ttest_ind(high_rank_averages, low_rank_averages, nan_policy='omit')

    # Store the result
    t_test_results.append((characteristic, t_stat, p_value))

# Print the T-test results for each characteristic
for characteristic, t_stat, p_value in t_test_results:
    print(f"Characteristic: {characteristic}")
    print(f"  T-statistic: {t_stat}, P-value: {p_value}")
    print()

Characteristic: Age_difference
  T-statistic: -2.1864064121651516, P-value: 0.04024883392203071

Characteristic: Film_count_difference
  T-statistic: -0.23629816072212692, P-value: 0.815490838854375

Characteristic: Average_revenue_difference
  T-statistic: -1.4360466558533724, P-value: 0.16571915181823657

Characteristic: Number_of_films_together
  T-statistic: -0.24008438776996235, P-value: 0.8125929006918391

Characteristic: Same_genre
  T-statistic: -0.5321586386412627, P-value: 0.6002017349026971



In [11]:
import dash
from dash import dcc, html
import plotly.graph_objs as go
import numpy as np
from tqdm import tqdm

# Function to calculate the average of a characteristic for a cluster
def calculate_average_for_cluster(cluster, df, characteristic):
    cluster_rows = df[(df['Actor1'].isin(cluster)) | (df['Actor2'].isin(cluster))]
    return cluster_rows[characteristic].mean()

# Prepare data for the bar chart
sorted_cluster_ids = sorted(cluster_averages, key=lambda x: cluster_averages[x]['average_rank'] or float('inf'))
cluster_ids = [f"Cluster {cluster_id}" for cluster_id in sorted_cluster_ids]
average_age_differences = []
hover_texts = []

for cluster_id in tqdm(sorted_cluster_ids, desc="Calculating average age differences"):
    # Calculate average age difference for each cluster
    age_diff = calculate_average_for_cluster(large_clusters[cluster_id], df1, 'Age_difference')
    average_age_differences.append(age_diff if age_diff is not None else 0)

    # Prepare hover text
    rank_info = f"Rank: {round(cluster_averages[cluster_id]['average_rank'])}"
    age_diff_info = f"Average Age Difference: {age_diff:.2f} years" if age_diff is not None else "Data missing"
    hover_texts.append(f"{cluster_ids[sorted_cluster_ids.index(cluster_id)]} - {rank_info} - {age_diff_info}")

# Create the bar chart with hover text
bar_chart = go.Figure(data=[go.Bar(x=cluster_ids, y=average_age_differences, hovertext=hover_texts, name='Average Age Difference')])

# Update layout to show hoverinfo
bar_chart.update_traces(hoverinfo='text', hoverlabel=dict(namelength=-1))

# Save the bar chart as an HTML file
bar_chart.write_html("../plots/plot_q6.html")

# Initialize the Dash app
app = dash.Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Graph(id='bar-chart', figure=bar_chart)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8080)

Calculating average age differences: 100%|██████████| 23/23 [00:01<00:00, 20.84it/s]


# **ANSWER TO THE QUESTION**

**6. Are there specific collaborative subgraphs within the network that consistently yield successful movies, and what are the characteristics of these subgraphs?**

In our investigation to identify the key traits of successful actor clusters, we employed a series of t-tests to assess various attributes within our dataset, including gender disparity, the frequency of film collaborations between actor pairs, and several other factors. Notably, the only attribute that yielded a statistically significant result (with a p-value below 0.05) was the age difference between pairs of actors.

The accompanying plot organizes clusters based on their average ranks, with the more successful clusters (denoted by lower average ranks) positioned on the left, and the less successful ones (indicated by higher average ranks) on the right. It is important to emphasize that a lower average rank signifies greater success. The plot reveals a discernible trend: clusters with lower ranks tend to exhibit a marginally higher age disparity, suggesting a broader age range among members of these clusters. This variation in age could be indicative of the beneficial impact of seasoned actors, who potentially contribute greater experience and knowledge to the ensemble.